<a href="https://colab.research.google.com/github/yosshy2020/intro_git/blob/master/biginners_cn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

train = pd.read_csv('/content/drive/MyDrive/biginners/train.csv')
test = pd.read_csv('/content/drive/MyDrive/biginners/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/biginners/sample_submit.csv')
print('train.shape:', train.shape)
print('test.shape:', test.shape)

train.shape: (11900, 13)
test.shape: (5100, 12)


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11900 entries, 0 to 11899
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           11900 non-null  int64 
 1   age             11900 non-null  int64 
 2   workclass       11900 non-null  object
 3   fnlwgt          11900 non-null  int64 
 4   education       11900 non-null  object
 5   education-num   11900 non-null  int64 
 6   marital-status  11900 non-null  object
 7   occupation      11900 non-null  object
 8   relationship    11900 non-null  object
 9   race            11900 non-null  object
 10  sex             11900 non-null  object
 11  native-country  11900 non-null  object
 12  Y               11900 non-null  int64 
dtypes: int64(5), object(8)
memory usage: 1.2+ MB


In [4]:
all_data = pd.concat((train.loc[:,'age':'native-country'],
                      test.loc[:,'age':'native-country']))

print(all_data.shape)

(17000, 11)


In [5]:
all_data_df =  pd.get_dummies(all_data, columns=["workclass"])
all_data_df =  pd.get_dummies(all_data_df, columns=["education"])
all_data_df =  pd.get_dummies(all_data_df, columns=["marital-status"])
all_data_df =  pd.get_dummies(all_data_df, columns=["occupation"])
all_data_df =  pd.get_dummies(all_data_df, columns=["relationship"])
all_data_df =  pd.get_dummies(all_data_df, columns=["race"])
all_data_df =  pd.get_dummies(all_data_df, columns=["sex"], drop_first=True)
all_data_df =  pd.get_dummies(all_data_df, columns=["native-country"])

In [6]:
all_data_df

,age,fnlwgt,education-num,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_HS-grad,education_Masters,education_Prof-school,education_Some-college,marital-status_Divorced,marital-status_Married-civ-spouse,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_?,occupation_Adm-clerical,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_White,sex_Male,native-country_Mexico,native-country_Philippines,native-country_United-States
0,21,132652,10,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1
1,29,132652,9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
2,19,132652,13,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
3,17,132652,9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1
4,47,132652,10,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,19,132652,10,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
5096,23,132652,9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
5097,36,132652,5,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
5098,33,132652,11,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1


In [7]:
scaler = MinMaxScaler()
all_data_df[['age', 'fnlwgt', 'education-num']] = scaler.fit_transform(all_data_df[['age', 'fnlwgt', 'education-num']])
all_data_df

,age,fnlwgt,education-num,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_HS-grad,education_Masters,education_Prof-school,education_Some-college,marital-status_Divorced,marital-status_Married-civ-spouse,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_?,occupation_Adm-clerical,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_White,sex_Male,native-country_Mexico,native-country_Philippines,native-country_United-States
0,0.085106,1.0,0.538462,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1
1,0.255319,1.0,0.461538,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
2,0.042553,1.0,0.769231,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
3,0.000000,1.0,0.461538,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1
4,0.638298,1.0,0.538462,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,0.042553,1.0,0.538462,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
5096,0.127660,1.0,0.461538,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
5097,0.404255,1.0,0.153846,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
5098,0.340426,1.0,0.615385,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1


In [8]:
all_data_df.shape

(17000, 55)

In [9]:
train.Y

0        0
1        0
2        0
3        0
4        0
        ..
11895    0
11896    0
11897    0
11898    0
11899    1
Name: Y, Length: 11900, dtype: int64

In [10]:
train_X = all_data_df[:train.shape[0]]
test_X = all_data_df[train.shape[0]:]
train_Y= train.Y

In [11]:
kf = KFold(n_splits=4, shuffle= True, random_state=123)
tr_idx, va_idx = list(kf.split(train_X))[0]

In [12]:
tr_idx

array([    0,     1,     4, ..., 11897, 11898, 11899])

In [13]:
va_idx

array([    2,     3,     6, ..., 11879, 11886, 11896])

In [14]:
tr_x, va_x = train_X.iloc[tr_idx], train_X.iloc[va_idx]
tr_y, va_y = train_Y.iloc[tr_idx], train_Y.iloc[va_idx]

In [15]:
print(tr_x.shape)
print(tr_y.shape)
print(va_x.shape)
print(va_x.shape)

(8925, 55)
(8925,)
(2975, 55)
(2975, 55)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

In [17]:
model = Sequential()

model.add(Dense( 55, activation='sigmoid', input_dim=55))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 55)                3080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 56        
Total params: 3,136
Trainable params: 3,136
Non-trainable params: 0
_________________________________________________________________


In [18]:
result = model.fit(tr_x, tr_y,
                   epochs=5,
                   batch_size=100,
                   validation_data=(va_x, va_y),
                   verbose=1)

Epoch 1/5
90/90 [==============================] - 1s 7ms/step - loss: 0.2059 - accuracy: 0.6663 - val_loss: 0.1558 - val_accuracy: 0.7395
Epoch 2/5
90/90 [==============================] - 0s 2ms/step - loss: 0.1467 - accuracy: 0.7608 - val_loss: 0.1337 - val_accuracy: 0.8161
Epoch 3/5
90/90 [==============================] - 0s 2ms/step - loss: 0.1271 - accuracy: 0.8261 - val_loss: 0.1243 - val_accuracy: 0.8289
Epoch 4/5
90/90 [==============================] - 0s 2ms/step - loss: 0.1212 - accuracy: 0.8348 - val_loss: 0.1201 - val_accuracy: 0.8289
Epoch 5/5
90/90 [==============================] - 0s 2ms/step - loss: 0.1187 - accuracy: 0.8324 - val_loss: 0.1178 - val_accuracy: 0.8313


In [19]:
result = model.predict(test_X)

In [20]:
result

array([[0.7094549 ],
       [0.4591595 ],
       [0.02365655],
       ...,
       [0.36508602],
       [0.21369949],
       [0.6082219 ]], dtype=float32)

In [21]:
preds_cnn = (result > 0.5).astype(int)

In [22]:
preds_cnn

array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [23]:
preds_cnn.dtype

dtype('int64')

In [24]:
preds_cnn.shape

(5100, 1)

In [25]:
preds_cnn=preds_cnn.flatten() 
preds_cnn

array([1, 0, 0, ..., 0, 0, 1])

In [26]:
test["index"]

0        3873
1        3625
2        3028
3       13814
4       15398
        ...  
5095     6726
5096     1501
5097     6391
5098     7104
5099     4214
Name: index, Length: 5100, dtype: int64

In [27]:
submission

,3873,0
0,3625,0
1,3028,0
2,13814,0
3,15398,0
4,13244,0
...,...,...
5094,6726,0
5095,1501,0
5096,6391,0
5097,7104,0


In [28]:
solutioncn=pd.DataFrame(preds_cnn, columns = ["submission"])

In [29]:
solutioncn = pd.DataFrame({"index":test["index"], "Y":preds_cnn})

In [30]:
solutioncn

,index,Y
0,3873,1
1,3625,0
2,3028,0
3,13814,0
4,15398,1
...,...,...
5095,6726,0
5096,1501,0
5097,6391,0
5098,7104,0


In [32]:
solutioncn["Y"].value_counts()

0    3983
1    1117
Name: Y, dtype: int64

In [ ]:
# solution.to_csv('./submit3.csv', index=False,header=False)